In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Иницијализација на Web Driver за Selenium
driver = webdriver.Chrome()
driver.get("https://kosuli.mk")
wait = WebDriverWait(driver, 10)

# Дефинирање на речник, dictionary на категориите кои ги скрејпам со големи букви
menu_targets = {
    "Брендови": {"CASA MODA", "FF-FASHION", "VENTI"},
    "Кратки ракави": {"МОДЕРН ФИТ"},
}

# Празно dictionary за чување на УРЛ's
extracted_urls = {}

for top_menu, targets in menu_targets.items():
    # Се чека елементот да биде пронајден со hover 
    link = wait.until(EC.presence_of_element_located(
        (By.XPATH, f"//a[contains(normalize-space(), '{top_menu}')]")
    ))
    actions = ActionChains(driver)
    actions.move_to_element(link).perform()

    # Се чеза за елементот да биде видлив за да може да се лоцира
    submenu = wait.until(EC.presence_of_element_located(
        (By.XPATH, f"//li[a[contains(normalize-space(), '{top_menu}')]]/ul")
    ))

    # Се зачувуваат сите линкови од секоја категорија 
    submenu_anchors = submenu.find_elements(By.TAG_NAME, "a")

    for a in submenu_anchors:
        text = a.text.strip().upper()
        href = a.get_attribute("href")
        if text in targets and href:
            # Се зачувува во варијабла линкот 
            key = text.lower().replace(" ", "_")
            extracted_urls[key] = href

# Затварање на бровсерот односно исклучување на селениум
driver.quit()

# Зачувување на варијабли, главната варијбала страницата и потоа секое урл посебно
base_url = "https://kosuli.mk"
url_casa_moda = extracted_urls.get("casa_moda")
url_ff_fashion = extracted_urls.get("ff-fashion")
url_venti = extracted_urls.get("venti")
url_modern_fit = extracted_urls.get("модерн_фит")

# Принтање на секој линк
print("Casa Moda :", url_casa_moda)
print("FF-Fashion:", url_ff_fashion)
print("Venti     :", url_venti)
print("Modern Fit:", url_modern_fit)


Casa Moda : https://kosuli.mk/product-category/casa-moda/
FF-Fashion: https://kosuli.mk/product-category/f-fashion/
Venti     : https://kosuli.mk/product-category/venti/
Modern Fit: https://kosuli.mk/product-category/modern-fit/


In [5]:
import csv
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

# Сетирање на селениум веб драјверот и исклучување на сликите да не се вчутуваат за побрзо лоадирање
options = Options()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)

product_urls = []
page = 1

# Изминување на сите страници за да се извлечат линковите на сите продукти
while True:
    driver.get(f"{url_modern_fit}page/{page}/")
    time.sleep(2)

    links = driver.find_elements(By.CSS_SELECTOR, "ul.products li.product a.woocommerce-LoopProduct-link")
    if not links:
        break

    for a in links:
        href = a.get_attribute("href")
        if href and href not in product_urls:
            product_urls.append(href)

    try:
        driver.find_element(By.CSS_SELECTOR, "a.next.page-numbers")
        page += 1
    except NoSuchElementException:
        break

print(f"→ Found {len(product_urls)} products across {page} pages.")

# Скрејпање на податоците за сите продукти
results = []
for idx, href in enumerate(product_urls, start=1):
    driver.get(href)
    time.sleep(1)

    # Скрејпање на името
    name = driver.find_element(By.TAG_NAME, "h1").text.strip()

    # Скрејпање на цената
    try:
        price = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
    except NoSuchElementException:
        price = ""

    # Скрејпање на урлто на сликите
    try:
        img = driver.find_element(By.CSS_SELECTOR, "div.woocommerce-product-gallery__image img")
        image_url = img.get_attribute("data-src") or img.get_attribute("src")
    except NoSuchElementException:
        image_url = ""

    # Скрејпање на величините за сите продукти, доколку нема остава празно
    sizes = []
    try:
        descr = driver.find_element(By.ID, "tab-description")
        driver.execute_script("arguments[0].scrollIntoView(true);", descr)
        div = descr.find_element(By.XPATH, ".//div[contains(text(),'Достапни големини')]")
        ul = div.find_element(By.XPATH, "following-sibling::ul[1]")
        sizes = [li.text.strip() for li in ul.find_elements(By.TAG_NAME, "li") if li.text.strip()]
    except Exception:
        pass

    sizes_str = ", ".join(sizes)

    # Зачувување на податоците 
    print(f"[{idx}/{len(product_urls)}] {name} → Sizes: {sizes_str or '—'}")
    results.append({
        "Name": name,
        "Price": price,
        "Sizes": sizes_str,
        "Image_URL": image_url,
        "Product_URL": href
    })

# Креирање на пандас дејтафрејм
df = pd.DataFrame(results)

# Зачувување во цсв формат и нивно принтање во формат име и величини
df.to_csv("kosuli_modern_fit.csv", index=False, encoding="utf-8")

print(f"✔ Scraped {len(df)} products. Saved to kosuli_modern_fit.csv.")
driver.quit()


→ Found 10 products across 1 pages.
[1/10] FFM010 MODERN FIT SHORT SLEEVE → Sizes: L, XL, XXL
[2/10] FFM009 MODERN FIT SHORT SLEEVE → Sizes: XXL
[3/10] FFM008 MODERN FIT SHORT SLEEVE → Sizes: XXL, 4XL
[4/10] FFM007 MODERN FIT SHORT SLEEVE → Sizes: M, L, XL, XXL,  5XL
[5/10] FFM006 MODERN FIT SHORT SLEEVE → Sizes: XL, XXL
[6/10] FFM005 MODERN FIT SHORT SLEEVE → Sizes: M, L, XL, XXL, 3XL, 4XL
[7/10] FFM004 MODERN FIT SHORT SLEEVE → Sizes: XXL
[8/10] FFM003 MODERN FIT SHORT SLEEVE → Sizes: L,  XXL, XL,4XL, 5XL
[9/10] FFM002 MODERN FIT SHORT SLEEVE → Sizes: M,  XXL, 4XL
[10/10] FFM001 MODERN FIT SHORT SLEEVE → Sizes: XXL
✔ Scraped 10 products. Saved to kosuli_modern_fit.csv.
